In [11]:
import pickle
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
import pandas as pd
import numpy as np

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [65]:
train_path = '/home/LLM_Taxonomy/TaxonomyEnrichment/data/м/train_hypernyms.pickle'
with open(train_path, 'rb') as f:
    train = pickle.load(f)

test_path = '/home/LLM_Taxonomy/TaxonomyEnrichment/data/psychology/test_hypernyms.pickle'
with open(test_path, 'rb') as f:
    test = pickle.load(f)

In [66]:
train[0], test[10]

({'children': 'compulsory heterosexuality.n.1',
  'parents': 'queer.n.1',
  'grandparents': None,
  'case': 'predict_hypernym'},
 {'children': 'anisochromasia',
  'grandparents': None,
  'parents': 'psychiatry',
  'case': 'predict_hypernym'})

In [22]:
def add_definitions(elem):
    if elem['case'] == 'predict_hypernym':
        elem['child_def'] = wn.synset(elem['children'].replace(' ', '_')).definition()
      #  elem['parent_def'] = wn.synset(elem['parents']).definition()
    elif elem['case'] == 'predict_multiple_hypernyms':
        elem['child_def'] = wn.synset(elem['children'].replace(' ', '_')).definition()

    elif elem['case'] == 'simple_triplet_grandparent':
        elem['child_def'] = wn.synset(elem['children'].replace(' ', '_')).definition()
        elem['grandparent_def'] = wn.synset(elem['grandparents']).definition()
    elif elem['case'] == 'only_child_leaf':
        elem['grandparent_def'] = wn.synset(elem['grandparents']).definition()
        elem['parent_def'] = wn.synset(elem['parents']).definition()
    elif elem['case'] == 'simple_triplet_2parent':
        elem['1parent_def'] = wn.synset(elem['parents'][0]).definition()
        elem['2parent_def'] = wn.synset(elem['parents'][1]).definition()
    else:
        elem['parent_def'] = wn.synset(elem['parents']).definition()

def cut_lemmas(lemmas):
    max_lemmas = min(len(lemmas), 8)
    return lemmas[:max_lemmas]

max_len = 0
def add_lemmas(elem):
    global max_len
    if elem['case'] == 'predict_hypernym':
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['children'].replace(' ', '_')).lemmas()])
        elem['child_def'] = ', '.join(lemmas)
    elif elem['case'] == 'simple_triplet_grandparent':
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['children'].replace(' ', '_')).lemmas()])
        elem['child_def'] = ', '.join(lemmas)
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['grandparents']).lemmas()])
        elem['grandparent_def'] = ', '.join(lemmas)
    elif elem['case'] == 'only_child_leaf':
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['grandparents']).lemmas()])
        elem['grandparent_def'] = ', '.join(lemmas)

        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['parents']).lemmas()])
        elem['parent_def'] = ', '.join(lemmas)
    elif elem['case'] == 'simple_triplet_2parent':
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['parents'][0]).lemmas()])
        elem['1parent_def'] = ', '.join(lemmas)
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['parents'][1]).lemmas()])
        elem['2parent_def'] = ', '.join(lemmas)
    else:
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['parents']).lemmas()])
        elem['parent_def'] = ', '.join(lemmas)
    if len(lemmas) > max_len:
        max_len = len(lemmas)

In [27]:
for i, elem in enumerate(train):
    try:
        add_definitions(elem)
    except:
        print(i, elem)
        train.remove(elem)

In [29]:
counter = 0
for i, elem in enumerate(test):
    try:
        add_definitions(elem)
    except:
        print(i, elem)
        counter += 1
        test.remove(elem)

print(counter)

0


In [30]:
len(test), len(train)

(985, 7712)

In [31]:
for elem in test:
    a = (elem['child_def'])

In [32]:
train_out = '/home/LLM_Taxonomy/TaxonomyEnrichment/data/verb/train_hypernyms_multisense_def.pickle'
test_out = '/home/LLM_Taxonomy/TaxonomyEnrichment/data/verb/test_hypernyms_def.pickle'

with open(train_out, 'wb') as f:
    pickle.dump(train, f)

with open(test_out, 'wb') as f:
    pickle.dump(test, f)

In [8]:
def_path = '/home/LLM_Taxonomy/TaxonomyEnrichment/data/MAG_CS/definitions_last.pickle'

with open(def_path, 'rb') as f:
    defs_wiki = pickle.load(f)

gen_def_path = '/home/LLM_Taxonomy/TaxonomyEnrichment/data/MAG_CS/chatgpt_def_MAG_CS.tsv'

gen_defs = pd.read_csv(gen_def_path, sep='\t', header=None)

In [10]:
len(defs_wiki)

29484

In [69]:
for i, row in gen_defs.iterrows():

    term = row[0]
    definition = row[1]

    defs_wiki[term] = definition

In [9]:
len(defs_wiki) - len(gen_defs), len(gen_defs)

(23770, 5714)

In [70]:
for elem in train:
    child = elem['children'].split('.')[0]
    elem['child_def'] = defs_wiki[child][:3000]

for elem in test:
    child = elem['children'].split('.')[0]
    elem['child_def'] = defs_wiki[child][:3000]


In [71]:
lens = [len(elem['child_def']) for elem in train]

In [72]:
np.argmax(lens), np.max(lens)

(21, 2866)

In [73]:
np.histogram(lens)

(array([8471, 2951,  765,  108,   14,    3,    1,    0,    0,    4]),
 array([  19. ,  303.7,  588.4,  873.1, 1157.8, 1442.5, 1727.2, 2011.9,
        2296.6, 2581.3, 2866. ]))

In [64]:
train_out = '/home/LLM_Taxonomy/TaxonomyEnrichment/data/psychology/train_hypernyms_def.pickle'
test_out = '/home/LLM_Taxonomy/TaxonomyEnrichment/data/psychology/test_hypernyms_def.pickle'

with open(train_out, 'wb') as f:
    pickle.dump(train, f)

with open(test_out, 'wb') as f:
    pickle.dump(test, f)

In [17]:
for elem in test:
    if elem['case'] == 'predict_multiple_hypernyms':
        a = predict_multiple_parents_from_child(elem)

In [5]:
def clean_elem(elem, keys_to_remove_digits=["children"]):
    removes = set(keys_to_remove_digits)
    if not "changed" in elem.keys():
        for field in ["children", "parents", "grandparents", "brothers"]:
            if field in elem.keys():
                elem[field] = delete_techniqal(
                    elem[field], remove=(field in removes))
                elem["changed"] = True
    return elem


def delete_techniqal(elem, remove):
    if isinstance(elem, str):
        if ".n." in elem and remove:
            return elem.split(".")[0].replace("_", " ")
        else:
            return elem.replace("_", " ")

    elif isinstance(elem, list):
        new_words = []
        for word in elem:
            new_words.append(delete_techniqal(word, remove))
        return new_words

def predict_parent_from_child(elem):
    """
    Predict the hypernym for the word “spaniel”. Answer: (sporting dog)
    """
    keys_to_remove_digits = []
    if use_def_prompt:
        keys_to_remove_digits.append("children") # do not need numbers when provided definition
    if (not use_number_target) or (use_def_target):
        keys_to_remove_digits.append("parents")
    
    clean = clean_elem(elem, keys_to_remove_digits=keys_to_remove_digits)

    if use_def_prompt:
        transformed_term = (
            "hyponym: " + clean["children"] + " (" + elem["child_def"] + ") | hypernym:"
        )
    else:
        transformed_term = "hyponym: " + clean["children"] + " | hypernym:"

    if use_def_target:
        target = clean["parents"] + " (" + elem["parent_def"] + ")" + ","
    else:
        target = clean["parents"] + ","

    return transformed_term, target

In [19]:
te = dict(elem)

In [20]:
use_def_prompt = True
use_def_target = True
use_number_target = True

predict_parent_from_child(te)

('hyponym: light hour (the distance light travels in a vacuum in one hour; approximately one billion kilometers) | hypernym:',
 'astronomy unit (a linear unit used for astronomical distances),')

In [9]:
processed_term

'hypernym: optical instrument (an instrument designed to aid vision) | hyponyms:'